In [1]:
import tensorflow
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input,decode_predictions
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import auc

In [2]:
def model():
    model = VGG16(weights='imagenet')
    int_model = tensorflow.keras.Model(model.inputs, [model.layers[5].output, model.layers[17].output])
    return int_model


def features_for_img(img_path, int_model):
    img = plt.imread(img_path)[:, :, :3]
    img = tensorflow.image.resize(img, [224,224]).numpy()
    img = np.expand_dims(img, 0)
    
    low, high = int_model.predict(img)
    return low, high


def get_features_for_task(task, int_model, base_path):
    train_base_path = base_path + 'train/'
    base_path = base_path + 'test/'
    
    train_wugs_low = []
    train_wugs_high = []
    for i in range(1, 6):
        curr_path = train_base_path + str(i) + '_fin.png'
        low, high = features_for_img(curr_path, int_model)
        train_wugs_low.append(low)
        train_wugs_high.append(high)
        
    wugs_low = []
    wugs_high = []
    for i in range(1, 6):
        curr_path = base_path + 'in_' + str(i) + '_fin.png'
        low, high = features_for_img(curr_path, int_model)
        wugs_low.append(low)
        wugs_high.append(high)

    not_wugs_low_sdiff = []
    not_wugs_high_sdiff = []
    for i in range(1, 6):
        curr_path = base_path + 'out_sdiff_' + str(i) + '_fin.png'
        low, high = features_for_img(curr_path, int_model)
        not_wugs_low_sdiff.append(low)
        not_wugs_high_sdiff.append(high)
        
    not_wugs_low_ldiff = []
    not_wugs_high_ldiff = []
    for i in range(1, 6):
        curr_path = base_path + 'out_ldiff_' + str(i) + '_fin.png'
        low, high = features_for_img(curr_path, int_model)
        not_wugs_low_ldiff.append(low)
        not_wugs_high_ldiff.append(high)
        
    return train_wugs_low, train_wugs_high, wugs_low, wugs_high, not_wugs_low_sdiff, not_wugs_high_sdiff, not_wugs_low_ldiff, not_wugs_high_ldiff

def get_diff_to_mean_wug(train_wugs, wugs, not_wugs_sdiff, not_wugs_ldiff):
    mean_train_wugs = np.squeeze(np.array(train_wugs))
    mean_train_wugs = np.mean(mean_train_wugs, 0)
       
    wugs_diff = []
    for e in wugs:
        wugs_diff.append(np.sum(np.abs(e - mean_train_wugs)))
      
    not_wugs_sdiff_diff = []
    for e in not_wugs_sdiff:
        not_wugs_sdiff_diff.append(np.sum(np.abs(e - mean_train_wugs)))
        
    not_wugs_ldiff_diff = []
    for e in not_wugs_ldiff:
        not_wugs_ldiff_diff.append(np.sum(np.abs(e - mean_train_wugs)))
    
    return wugs_diff, not_wugs_sdiff_diff, not_wugs_ldiff_diff

def tpr_fpr_for_threshold(wug, not_wug, threshold):
    pos = 0
    for d in wug:
        if d <= threshold:
            pos += 1
    tpr = pos / len(wug)
    
    neg_wrong = 0
    for d in not_wug:
        if d <= threshold:
            neg_wrong += 1
    fpr = neg_wrong / len(not_wug)
    
    return tpr, fpr

def acc_for_threshold(wug, not_wug, threshold):
    total = 0
    pos = 0
    for d in wug:
        total += 1
        if d <= threshold:
            pos += 1
            
    neg = 0
    for d in not_wug:
        total += 1
        if d > threshold:
            neg += 1    
    return (pos + neg) / total


def get_max_acc(wugs_diff, not_wugs_diff):
    all_diffs = sorted(wugs_diff + not_wugs_diff)
    accs = []
    for t in all_diffs:
        accs.append(acc_for_threshold(wugs_diff, not_wugs_diff, t))
    accs = np.array(accs)
    return accs.max()


def get_auc(wugs_diff, not_wugs_diff):
    all_diffs = sorted(wugs_diff + not_wugs_diff)
    tprs = []
    fprs = []
    ts = all_diffs
    for t in ts:
        tpr, fpr = tpr_fpr_for_threshold(wugs_diff, not_wugs_diff, t)
        tprs.append(tpr)
        fprs.append(fpr)
    curr_auc = auc(fprs, tprs)
    return curr_auc, fprs, tprs
    

def eval_for_task(task, int_model, base_path):
    train_wugs_low, train_wugs_high, wugs_low, wugs_high, not_wugs_low_sdiff, not_wugs_high_sdiff, not_wugs_low_ldiff, not_wugs_high_ldiff = get_features_for_task(task, int_model, base_path)
    
    # Low level features
    wugs_low_diff, not_wugs_sdiff_low_diff, not_wugs_ldiff_low_diff = get_diff_to_mean_wug(train_wugs_low, wugs_low, not_wugs_low_sdiff, not_wugs_low_ldiff)
    
    auc_low, _, _, = get_auc(wugs_low_diff, not_wugs_sdiff_low_diff + not_wugs_ldiff_low_diff)
    auc_low_sdiff, _, _, = get_auc(wugs_low_diff, not_wugs_sdiff_low_diff)
    auc_low_ldiff, _, _, = get_auc(wugs_low_diff, not_wugs_ldiff_low_diff)
    
    acc_low = get_max_acc(wugs_low_diff, not_wugs_sdiff_low_diff + not_wugs_ldiff_low_diff)
    acc_low_sdiff = get_max_acc(wugs_low_diff, not_wugs_sdiff_low_diff)
    acc_low_ldiff = get_max_acc(wugs_low_diff, not_wugs_ldiff_low_diff)
    
    # High level features
    wugs_high_diff, not_wugs_sdiff_high_diff, not_wugs_ldiff_high_diff = get_diff_to_mean_wug(train_wugs_high, wugs_high, not_wugs_high_sdiff, not_wugs_high_ldiff)
    
    auc_high, _, _, = get_auc(wugs_high_diff, not_wugs_sdiff_high_diff + not_wugs_ldiff_high_diff)
    auc_high_sdiff, _, _, = get_auc(wugs_high_diff, not_wugs_sdiff_high_diff)
    auc_high_ldiff, _, _, = get_auc(wugs_high_diff, not_wugs_ldiff_high_diff)
    
    acc_high = get_max_acc(wugs_high_diff, not_wugs_sdiff_high_diff + not_wugs_ldiff_high_diff)
    acc_high_sdiff = get_max_acc(wugs_high_diff, not_wugs_sdiff_high_diff)
    acc_high_ldiff = get_max_acc(wugs_high_diff, not_wugs_ldiff_high_diff)
    
    print('AUC low-level features: ' + str(round(auc_low, 4)) + '\n')
    print('AUC low-level features: (sdiff): ' + str(round(auc_low_sdiff, 4)) + '\n')
    print('AUC low-level features (ldiff): ' + str(round(auc_low_ldiff, 4)) + '\n')
    print('\n')
    print('AUC high-level features: ' + str(round(auc_high, 4)) + '\n')
    print('AUC high-level features (sdiff): ' + str(round(auc_high_sdiff, 4)) + '\n')
    print('AUC high-level features (ldiff): ' + str(round(auc_high_ldiff, 4)) + '\n')
    print('\n')
    print('Max accuracy low-level features: ' + str(round(acc_low, 4)) + '\n')
    print('Max accuracy low-level features (sdiff): ' + str(round(acc_low_sdiff, 4)) + '\n')
    print('Max accuracy low-level features (ldiff): ' + str(round(acc_low_ldiff, 4)) + '\n')
    print('\n')
    print('Max accuracy high-level features: ' + str(round(acc_high, 4)) + '\n')
    print('Max accuracy high-level features (sdiff): ' + str(round(acc_high_sdiff, 4)) + '\n')
    print('Max accuracy high-level features (ldiff): ' + str(round(acc_high_ldiff, 4)) + '\n')

    



In [3]:
int_model = model()

2022-06-05 12:23:54.007407: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-05 12:23:54.467298: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22311 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:41:00.0, compute capability: 8.6


In [4]:
eval_for_task('concept_cct', int_model, '/viscam/u/joycj/geoclidean/dataset/z_geo_dataset/constraints/concept_cct/')





2022-06-05 12:23:56.856935: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2022-06-05 12:23:57.486530: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


AUC low-level features: 0.48

AUC low-level features: (sdiff): 0.44

AUC low-level features (ldiff): 0.52



AUC high-level features: 0.78

AUC high-level features (sdiff): 0.68

AUC high-level features (ldiff): 0.88



Max accuracy low-level features: 0.6667

Max accuracy low-level features (sdiff): 0.6

Max accuracy low-level features (ldiff): 0.7



Max accuracy high-level features: 0.8

Max accuracy high-level features (sdiff): 0.7

Max accuracy high-level features (ldiff): 0.8

